In [0]:
import pandas as pd
import sqlite3 as sql

# Conectar ao banco SQLite3
conn = sql.connect('data/pipeline.db')
print("Conexão com banco estabelecida")

# ==========================================
# CARREGAR DADOS AGREGADOS (GOLD)
# ==========================================
country = pd.read_parquet('data/gold/dim_country.parquet')
country.to_sql('dim_country', conn, if_exists='replace', index=False)

customer = pd.read_parquet('data/gold/dim_customer.parquet')
customer.to_sql('dim_customer', conn, if_exists='replace', index=False)

date = pd.read_parquet('data/gold/dim_date.parquet') 
date.to_sql('dim_date', conn, if_exists='replace', index=False)

product = pd.read_parquet('data/gold/dim_product.parquet')
product.to_sql('dim_product', conn, if_exists='replace', index=False) 

fact = pd.read_parquet('data/gold/fact_all.parquet')
fact.to_sql('fact', conn, if_exists='replace', index=False)

# ==========================================
# VERIFICAR TABELAS CRIADAS
# =========================================
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("\nTabelas no banco de dados:")

for tabela in tabelas:
    print(f"  - {tabela[0]}")

conn.close()
print("\nDados carregados com sucesso no banco de dados!")

Conexão com banco estabelecida


539273

In [0]:
# Consulta SQL para ver o total de vendas na tabela fact
conn = sql.connect('data/pipeline.db')
query = "SELECT SUM(total_value) AS total_vendas FROM fact;"
total_vendas = pd.read_sql_query(query, conn)
print(f"Total de vendas: {total_vendas.iloc[0,0]:,.2f}")
conn.close()

Total de vendas: 9,759,687.46
